<p style="font-size:150%">ПИ20-2 Лим А. зачет</p>

Вариант 10

Вопрос 1. 

Создание таблиц и выполнение SQL-запросов с помощью библиотеки SQLite3 Python.

Прежде чем еобходимо импортировать библиотеку с помощью `import sqlite3`. 

Чтобы создать новую базу данных SQLite нужно создать подключение к ней (файл БД будет создан автоматически). Для этого создаем объект Connection с помощью функции `connect()`. Если требуется создать файл базы данных в конкретной папке, необходимо в месте с именем файла указать путь к ней, например: ```"D:/Data/pythonsqlite.db"```.

Чтобы создать новую таблицу в базе данных SQLite, выполним следующие действия:

1. Напишем выражения, обеспечивающие создание таблиц на языке SQL.
2. Создадим объект *Connection*, используя функцию `connect()` *sqlite3*.
3. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
4. Передадим SQL-выражение методу `execute()` объекта *Cursor* и выполним этот метод.

Вопрос 2. 

С помощью библиотеки SQLAlchemy создайте объекты таблиц базы данных Chinook. В таблице Employee содержится информация о сотрудниках, в таблице Customer - о клиентах, в таблице Invoice - информация о заказах. Информация о сотруднике, обслужившем конкретного клиента в поле SupportRepId. Составьте запрос, показывающий, сумму продаж каждого сотрудника в 2010 году. Результат должен содержать имя, фамилию сотрудника, сумму продаж.

In [2]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text, desc, union, union_all, asc
from sqlalchemy.sql import func, exists, text, distinct
import pandas as pd

In [12]:
engine = create_engine('sqlite:///../Data/Chinook_Sqlite.sqlite')
metadata = MetaData()
metadata.reflect(bind = engine)
conn = engine.connect()

In [13]:
metadata.tables.keys()

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'Track', 'MediaType', 'Playlist', 'PlaylistTrack'])

In [14]:
Album=Table("Album", metadata, autoload=True, autoload_with="engine")
Artist=Table("Artist", metadata, autoload=True, autoload_with="engine")
Customer=Table("Customer", metadata, autoload=True, autoload_with="engine")
Employee=Table("Employee", metadata, autoload=True, autoload_with="engine")
Genre=Table("Genre", metadata, autoload=True, autoload_with="engine")
Invoice=Table("Invoice", metadata, autolad=True, autolad_with="engine")
InvoiceLine=Table("InvoiceLine", metadata, autoload=True, autoload_with="engine")
Track=Table("Track", metadata, autoload=True, autoload_with="engine")
MediaType=Table("MediaType", metadata, autoload=True, autoload_with="engine")
Playlist=Table("Playlist", metadata, autoload=True, autoload_with="engine")
PlaylistTrack=Table("PlaylistTrack", metadata, autoload=True, autoload_with="engine")

In [15]:
subq = select(Employee.c.EmployeeId, Employee.c.LastName, Employee.c.FirstName, Invoice.c.Total )\
    .join_from(Employee, Customer, Employee.c.EmployeeId == Customer.c.SupportRepId)\
    .join(Invoice, Customer.c.CustomerId == Invoice.c.CustomerId).subquery()

stmt = select(subq.c.LastName, subq.c.FirstName, func.sum(subq.c.Total).label('Сумма')).group_by(subq.c.EmployeeId)

df = pd.read_sql(stmt, conn)
print(df)

  LastName FirstName   Сумма
0  Peacock      Jane  833.04
1     Park  Margaret  775.40
2  Johnson     Steve  720.16


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:1402: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return self.connectable.execution_options().execute(*args, **kwargs)


Вопрос 3. 

Набор данных BikeShare_19_20.csv содержит информацию о поездках между станциями проката велосипедов. С помощью библиотеки SQLAlchemy cоздайте таблицу БД и вставьте в эту таблицу данные из файла BikeShare_19_20.csv. Решите задачу.
Атрибут member_casual содержит тип клиента. Отдельно для каждого типа клиента покажите разность в секундах продолжительности каждой поездки и средней продолжительности поездки среди поездок клиентов данного типа.

In [7]:
engine = create_engine('sqlite+pysqlite:///../Data/BikeShareService.sqlite')
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)

In [8]:
sql = '''
CREATE TABLE BikeShare (
    ride_id text PRIMARY KEY,
    rideable_type text,
    started_at text,
    ended_at text,
    start_station_name text,
    start_station_id integer,
    end_station_name text,
    end_station_id integer,
    start_lat numeric,
    start_lng numeric,
    end_lat numeric,
    end_lng numeric,
    member_casual text,
    duration integer);
'''
conn.execute(sql)

In [9]:
path_to_csv = '../Data/BikeShare_19_20.csv'
df = pd.read_csv(path_to_csv)
df.to_sql('BikeShare', con=conn, if_exists = 'append', index=False)

457

In [10]:
stmt = """
WITH subq AS(
    SELECT ride_id, member_casual, duration,
    avg(duration) over(partition by member_casual) AS Avg
    FROM BikeShare
),
subq2 AS(
    SELECT ride_id, member_casual, duration, round(Avg, 2) AS Average_duration
    FROM subq
)
SELECT *, duration-Average_duration AS Difference
FROM subq2
"""

df = pd.read_sql(stmt, conn)
print(df)

              ride_id member_casual  duration  Average_duration  Difference
0    D4D62DFDE089698B        casual       536           5244.92    -4708.92
1    16EA6FB99FB215E8        casual      5681           5244.92      436.08
2    DF27A9FB30E2FD63        casual      6195           5244.92      950.08
3    2A74CFCD2162E111        casual      1128           5244.92    -4116.92
4    483CB7BD2969A1BE        casual      4654           5244.92     -590.92
..                ...           ...       ...               ...         ...
452  2B45E461C98F0BB4        member      1797           1149.08      647.92
453  31D96A503057EBEE        member      3690           1149.08     2540.92
454  C2D07E36E4185198        member      1028           1149.08     -121.08
455  7E04BBAE91B22004        member       597           1149.08     -552.08
456  2411BE3F9BF28476        member      1522           1149.08      372.92

[457 rows x 5 columns]
